In [1]:
import Collobrative_Filtering as CF
import numpy as np
import pandas as pd
import redis
import json
import random
import pickle
import inspect
import os

In [2]:
redis_conn = redis.Redis(host='localhost',port=6379,db=0)

## 測試目的：使用演算法的話是否成效會比Random更好

In [3]:
model = CF.COLLOBORATIVE_FILTERING()

In [4]:
with open('tag_mapping.pickle', 'rb') as f3:
    tag_mapping = pickle.load(f3)

In [5]:
def preference_fake(data):
    number = len(data)
    random_lst = []
    for i in range(number):
        if random.randint(0,1)==1:
            random_lst.append(i)
    return random_lst

In [6]:
def preference(number):
    ran_num = random.randint(1,number)
    data = json.loads(redis_conn.get(ran_num))
    model_tags = tag_mapping.keys()
    user_tags = []
    for i in data['batchTag']:
        if i[u'TAG_ID'] in model_tags:
            user_tags.append(tag_mapping[i[u'TAG_ID']])
    return user_tags

## Model Test

In [ ]:
for j in range(1,1501):
    data = json.loads(redis_conn.get(j))
    model_tags = tag_mapping.keys()
    user_tags = []
    for i in data['batchTag']:
        if i[u'TAG_ID'] in model_tags:
            user_tags.append(tag_mapping[i[u'TAG_ID']])
    test = model.predict(user_tags,False,similar_num=5,offertag_num=15,offer_number=3)
    print test

In [ ]:
data = json.loads(redis_conn.get(2))
model_tags = tag_mapping.keys()
user_tags = []
for i in data['batchTag']:
    if i[u'TAG_ID'] in model_tags:
        user_tags.append(tag_mapping[i[u'TAG_ID']])
test = model.predict(user_tags,False,similar_num=5,offertag_num=10,offer_number=3)
print test

In [ ]:
customer_tag = preference_fake(tag_mapping)
customer_reco = model.predict(customer_tag,False,similar_num=10,offertag_num=15,offer_number=3)
customer_reco

## 情境一：視每個顧客為新顧客（描述顧客標籤隨機給予）

### 顧客初始不喜歡推薦，且推薦各自獨立，Swich推薦的比例

In [ ]:
with open('offerlabel_mapping.pickle', 'rb') as f2:
    offerlabel_mapping = pickle.load(f2)
offer_tags = pd.read_csv('OFFERTAG_MAPPING.csv')

In [ ]:
def input_data_case1(number, tag_mapping):
    customer = np.zeros(number).tolist()
    for i in range(number):
        cust_tag = preference(1500)
        #tag = np.zeros(len(tag_mapping))
        #tag[[cust_tag]] = 1
        customer[i] = cust_tag
    return customer

In [ ]:
def simulation_case1(cust_tag, times, offer_tags, offerlabel_mapping, tag_mapping):
    response = np.zeros([len(cust_tag), times, 2]).tolist()
    for i in range(times):
        for j in range(len(cust_tag)):
            customer_reco = model.predict(cust_tag[j], False, similar_num=10, offertag_num=15, offer_number=3)
            label = offer_tags[offer_tags.OFFER_ID.isin(customer_reco)].LABEL_ID.unique()

            # 令3個被推薦的點擊機率均為0.1，且互相獨立
            if random.random()>=(0.9)**3:
                reco = customer_reco[random.randint(0,2)]
                label = offer_tags[offer_tags.OFFER_ID.isin([reco])].LABEL_ID.unique()
                offerlabel_map = {}
                offerlabel = np.zeros(len(offerlabel_mapping))
                for r in offerlabel_mapping.keys():
                    offerlabel_map[offerlabel_mapping[r]]=r
                for k in label:
                    offerlabel[offerlabel_map[k]] = 1
                rating_table = model.fit(cust_tag[j],offerlabel,True)
                response[j][i][0] = ','.join(customer_reco)
                response[j][i][1] = reco
                print 'Finish ' + str(j+1) + ' customer ' + str(i+1) + ' times!!' 

            else:

                response[j][i][0] = ','.join(customer_reco)
                print 'Finish ' + str(j+1) + ' customer ' + str(i+1) + ' times!!'  
    return response

In [ ]:
sim_data = input_data(100, tag_mapping)

In [ ]:
result = simulation_case1(sim_data,10,offer_tags,offerlabel_mapping,tag_mapping)

## 情境二：視每個顧客為新顧客（描述顧客標籤固定）

### 顧客初始有喜好分配，推薦內容各自獨立，其在模型運算後最後跑到他所喜好的產品機率

顧客 ID : 9, 10, 38, 48, 57, 71, 72, 78, 86, 90

In [7]:
# Load Customer's Truely Preference
with open('Preference.pickle', 'rb') as f:
    Preference_Dist = pickle.load(f)
with open('IndexMapOffer.pickle', 'rb') as f1:
    IndexMapOffer = pickle.load(f1)
with open('offerlabel_mapping.pickle', 'rb') as f2:
    offerlabel_mapping = pickle.load(f2)
offer_tags = pd.read_csv('OFFERTAG_MAPPING.csv')

In [8]:
# 將顧客資訊萃取出來
cust_id = [9, 10, 38, 48, 57, 71, 72, 78, 86, 90]
test_data = []
for j in cust_id:
    data = json.loads(redis_conn.get(j))
    model_tags = tag_mapping.keys()
    user_tags = []
    for i in data['batchTag']:
        if i[u'TAG_ID'] in model_tags:
            user_tags.append(tag_mapping[i[u'TAG_ID']])
    test_data.append(user_tags)

In [9]:
# 現實生活上的點擊率
answer = np.zeros(len(Preference_Dist)).tolist()
for data in range(len(Preference_Dist)):
    answer[data] = 1 - ((1 - sorted(Preference_Dist[data],reverse=True)[0])*
                    (1 - sorted(Preference_Dist[data],reverse=True)[1])*
                    (1 - sorted(Preference_Dist[data],reverse=True)[2]))

In [10]:
def simulation_case2(cust_tag, times, offer_tags, offerlabel_mapping, tag_mapping, Preference_Dist, IndexMapOffer, answer):
    response = np.zeros([len(cust_tag), times, 4]).tolist()
    recommand_items = np.ones(len(cust_tag))
    reco_distribution = np.zeros([len(cust_tag),len(IndexMapOffer)])
    for i in range(times):
        for j in range(len(cust_tag)):
            if recommand_items[j] > 13:
                recommand_items[j] = 1
            customer_reco = model.predict(cust_tag[j], False, similar_num=10, offertag_num=15, offer_number= int(3*recommand_items[j]))[-3:]
            Preference_lst = []
            NonClick_Rate_tot = 1
            recommand_items[j] += 1 
            for data in customer_reco:
                Preference_lst.append(Preference_Dist[j][IndexMapOffer[data]])
                NonClick_Rate_tot *= 1 - Preference_Dist[j][IndexMapOffer[data]]
                reco_distribution[j][IndexMapOffer[data]] += 1
            Preference_lst = np.array(Preference_lst)/np.array(Preference_lst).sum()    
            Click_Rate_tot = 1- NonClick_Rate_tot
            print Click_Rate_tot
            print Preference_lst
                            
            # 使用Preference的資訊
            if random.random()>=NonClick_Rate_tot:
                #Choose item
                choose = random.random()
                for item_num in range(len(Preference_lst)):
                    if  Preference_lst[:item_num].sum()<=choose<=Preference_lst[:item_num+1].sum():
                        reco = customer_reco[item_num]
                        reco_labels = offer_tags[offer_tags.OFFER_ID.isin([reco])].LABEL_ID.unique()
                        offerlabel_map = {}
                        offerlabel = np.zeros(len(offerlabel_mapping))

                        for key in offerlabel_mapping.keys():
                            offerlabel_map[offerlabel_mapping[key]]=key
                        for label in reco_labels:
                            offerlabel[offerlabel_map[label]] = 1
                        rating_table = model.fit(cust_tag[j],offerlabel,True)                    
                        response[j][i][1] = reco
                        recommand_items[j] = 1
                        print 'The ' + str(j+1) + ' customer clicked ' + str(item_num+1) + ' recommand item!!'
                        break
                                                                 
            response[j][i][0] = ','.join(customer_reco)
            response[j][i][2] = Click_Rate_tot
            response[j][i][3] = (Click_Rate_tot - answer[j])**2
            print 'Finish ' + str(j+1) + ' customer ' + str(i+1) + ' times!!'  
                    
    return (response , reco_distribution)

In [11]:
result = simulation_case2(test_data,100,offer_tags,offerlabel_mapping,tag_mapping,Preference_Dist, IndexMapOffer,answer)

0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 1 customer 1 times!!
0.975249375
[ 0.98984772  0.00507614  0.00507614]
The 2 customer clicked 1 recommand item!!
Finish 2 customer 1 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 3 customer 1 times!!
0.20798
[ 0.95238095  0.02380952  0.02380952]
Finish 4 customer 1 times!!
0.20798
[ 0.95238095  0.02380952  0.02380952]
Finish 5 customer 1 times!!
0.20798
[ 0.02380952  0.02380952  0.95238095]
Finish 6 customer 1 times!!
0.801995
[ 0.98765432  0.00617284  0.00617284]
Finish 7 customer 1 times!!
0.20798
[ 0.95238095  0.02380952  0.02380952]
Finish 8 customer 1 times!!
0.801995
[ 0.98765432  0.00617284  0.00617284]
The 9 customer clicked 1 recommand item!!
Finish 9 customer 1 times!!
0.975249375
[ 0.98984772  0.00507614  0.00507614]
The 10 customer clicked 1 recommand item!!
Finish 10 customer 1 times!!
0.20798
[ 0.95238095  0.02380952  0.02380952]
Finish 1 customer 2 times!!
0.975249375
[ 0.98984772  0.00507614  0.

The 10 customer clicked 1 recommand item!!
Finish 10 customer 10 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 1 customer 11 times!!
0.975249375
[ 0.98984772  0.00507614  0.00507614]
The 2 customer clicked 1 recommand item!!
Finish 2 customer 11 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 3 customer 11 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 4 customer 11 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 5 customer 11 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 6 customer 11 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 7 customer 11 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 8 customer 11 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 9 customer 11 times!!
0.975249375
[ 0.98984772  0.00507614  0.00507614]
The 10 customer clicked 1 recommand item!!
Finish 10 customer 11 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
F

The 2 customer clicked 1 recommand item!!
Finish 2 customer 20 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 3 customer 20 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 4 customer 20 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 5 customer 20 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 6 customer 20 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 7 customer 20 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 8 customer 20 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 9 customer 20 times!!
0.975249375
[ 0.98984772  0.00507614  0.00507614]
The 10 customer clicked 1 recommand item!!
Finish 10 customer 20 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 1 customer 21 times!!
0.975249375
[ 0.98984772  0.00507614  0.00507614]
The 2 customer clicked 1 recommand item!!
Finish 2 customer 21 times!!
0.20798
[ 0.02380952  0.02380952  0.95238095]
Finish 

The 1 customer clicked 1 recommand item!!
Finish 1 customer 30 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 2 customer 30 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 3 customer 30 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 4 customer 30 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 5 customer 30 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 6 customer 30 times!!
0.801995
[ 0.98765432  0.00617284  0.00617284]
The 7 customer clicked 1 recommand item!!
Finish 7 customer 30 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 8 customer 30 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 9 customer 30 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 10 customer 30 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 1 customer 31 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 2 customer 31 times!!
0.014925125
[ 0.3333

The 10 customer clicked 1 recommand item!!
Finish 10 customer 40 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 1 customer 41 times!!
0.975249375
[ 0.98984772  0.00507614  0.00507614]
The 2 customer clicked 1 recommand item!!
Finish 2 customer 41 times!!
0.3632
[ 0.49382716  0.01234568  0.49382716]
Finish 3 customer 41 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 4 customer 41 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 5 customer 41 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 6 customer 41 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 7 customer 41 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 8 customer 41 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 9 customer 41 times!!
0.975249375
[ 0.98984772  0.00507614  0.00507614]
The 10 customer clicked 1 recommand item!!
Finish 10 customer 41 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish

0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 8 customer 50 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 9 customer 50 times!!
0.975249375
[ 0.98984772  0.00507614  0.00507614]
The 10 customer clicked 1 recommand item!!
Finish 10 customer 50 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 1 customer 51 times!!
0.975249375
[ 0.98984772  0.00507614  0.00507614]
The 2 customer clicked 1 recommand item!!
Finish 2 customer 51 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 3 customer 51 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 4 customer 51 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 5 customer 51 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 6 customer 51 times!!
0.801995
[ 0.98765432  0.00617284  0.00617284]
The 7 customer clicked 1 recommand item!!
Finish 7 customer 51 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 8 customer 51 times!!
0.0149

The 10 customer clicked 1 recommand item!!
Finish 10 customer 60 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 1 customer 61 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 2 customer 61 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 3 customer 61 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 4 customer 61 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 5 customer 61 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 6 customer 61 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 7 customer 61 times!!
0.034725625
[ 0.14285714  0.14285714  0.71428571]
Finish 8 customer 61 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 9 customer 61 times!!
0.975249375
[ 0.98984772  0.00507614  0.00507614]
The 10 customer clicked 1 recommand item!!
Finish 10 customer 61 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 1 customer 62 times!!
0.014925125
[ 

The 9 customer clicked 1 recommand item!!
Finish 9 customer 70 times!!
0.975249375
[ 0.98984772  0.00507614  0.00507614]
The 10 customer clicked 1 recommand item!!
Finish 10 customer 70 times!!
0.20798
[ 0.02380952  0.02380952  0.95238095]
Finish 1 customer 71 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 2 customer 71 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 3 customer 71 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 4 customer 71 times!!
0.20798
[ 0.95238095  0.02380952  0.02380952]
Finish 5 customer 71 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 6 customer 71 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 7 customer 71 times!!
0.20798
[ 0.95238095  0.02380952  0.02380952]
Finish 8 customer 71 times!!
0.801995
[ 0.98765432  0.00617284  0.00617284]
Finish 9 customer 71 times!!
0.975249375
[ 0.98984772  0.00507614  0.00507614]
The 10 customer clicked 1 recommand item!!
Finish 10 custome

0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 9 customer 81 times!!
0.975249375
[ 0.98984772  0.00507614  0.00507614]
The 10 customer clicked 1 recommand item!!
Finish 10 customer 81 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 1 customer 82 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 2 customer 82 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 3 customer 82 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 4 customer 82 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 5 customer 82 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 6 customer 82 times!!
0.801995
[ 0.98765432  0.00617284  0.00617284]
The 7 customer clicked 1 recommand item!!
Finish 7 customer 82 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 8 customer 82 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 9 customer 82 times!!
0.975249375
[ 0.98984772  0.00507614  0.00507614

The 2 customer clicked 1 recommand item!!
Finish 2 customer 92 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 3 customer 92 times!!
0.20798
[ 0.95238095  0.02380952  0.02380952]
Finish 4 customer 92 times!!
0.20798
[ 0.95238095  0.02380952  0.02380952]
Finish 5 customer 92 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 6 customer 92 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 7 customer 92 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 8 customer 92 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 9 customer 92 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 10 customer 92 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 1 customer 93 times!!
0.975249375
[ 0.98984772  0.00507614  0.00507614]
The 2 customer clicked 1 recommand item!!
Finish 2 customer 93 times!!
0.014925125
[ 0.33333333  0.33333333  0.33333333]
Finish 3 customer 93 times!!
0.014925125
[ 0.33333333 

In [15]:
result[1]

array([[  7.,   7.,  11.,   9.,  10.,  11.,  11.,  10.,   7.,   7.,   7.,
          7.,   7.,   7.,   7.,   7.,   7.,   7.,   8.,   7.,   8.,  11.,
          7.,   8.,   7.,   7.,   7.,   7.,   7.,   7.,   7.,   7.,   7.,
          7.,   7.,   7.,   7.,   7.,   7.],
       [  3.,   3.,  64.,  64.,   3.,   3.,  64.,   3.,   3.,   3.,   3.,
          3.,   3.,   3.,   3.,   3.,   3.,   3.,   3.,   3.,   3.,   3.,
          3.,   3.,   3.,   3.,   3.,   3.,   3.,   3.,   3.,   3.,   3.,
          3.,   3.,   3.,   3.,   3.,   3.],
       [ 11.,   6.,  12.,  12.,  11.,  12.,  10.,  11.,  12.,   9.,   9.,
          6.,   9.,   6.,   6.,   9.,  11.,   6.,   6.,   6.,   6.,   6.,
          9.,   9.,   6.,   6.,   6.,   6.,   6.,   6.,   6.,   6.,   6.,
          6.,   6.,   6.,   6.,   6.,   6.],
       [  8.,   8.,   8.,   8.,   8.,   8.,   8.,   8.,   8.,   8.,   8.,
          8.,   8.,   7.,   8.,   8.,   8.,   7.,   8.,   7.,   8.,   8.,
          8.,   8.,   7.,   7.,   7.,   7.,   7.,  

In [18]:
recommand_distribution = np.zeros(result[1].shape)
for i in range(len(result[1])):
    recommand_distribution[i] = result[1][i]/result[1][i].sum()
recommand_distribution

array([[ 0.02333333,  0.02333333,  0.03666667,  0.03      ,  0.03333333,
         0.03666667,  0.03666667,  0.03333333,  0.02333333,  0.02333333,
         0.02333333,  0.02333333,  0.02333333,  0.02333333,  0.02333333,
         0.02333333,  0.02333333,  0.02333333,  0.02666667,  0.02333333,
         0.02666667,  0.03666667,  0.02333333,  0.02666667,  0.02333333,
         0.02333333,  0.02333333,  0.02333333,  0.02333333,  0.02333333,
         0.02333333,  0.02333333,  0.02333333,  0.02333333,  0.02333333,
         0.02333333,  0.02333333,  0.02333333,  0.02333333],
       [ 0.01      ,  0.01      ,  0.21333333,  0.21333333,  0.01      ,
         0.01      ,  0.21333333,  0.01      ,  0.01      ,  0.01      ,
         0.01      ,  0.01      ,  0.01      ,  0.01      ,  0.01      ,
         0.01      ,  0.01      ,  0.01      ,  0.01      ,  0.01      ,
         0.01      ,  0.01      ,  0.01      ,  0.01      ,  0.01      ,
         0.01      ,  0.01      ,  0.01      ,  0.01      ,  0.

In [17]:
finalresult = []
for i in range(len(result[0])):
    finalresult.append(result[0][i][-1])
finalresult

[['OFF0001,OFF0005,OFF0027', 0.0, 0.014925124999999984, 0.037270184761265611],
 ['OFF0004,OFF0005,OFF0008', 0.0, 0.97524937499999997, 0.0],
 ['OFF0008,OFF0001,OFF0020', 0.0, 0.20797999999999994, 0.078411200399999964],
 ['OFF0019,OFF0041,OFF0014', 0.0, 0.014925124999999984, 0.037270184761265611],
 ['OFF0007,OFF0008,OFF0022', 0.0, 0.20797999999999994, 0.59616929439999999],
 ['OFF0034,OFF0036,OFF0037', 0.0, 0.014925124999999984, 0.037270184761265611],
 ['OFF0028,OFF0029,OFF0031', 0.0, 0.014925124999999984, 0.61947898813251567],
 ['OFF0009,OFF0001,OFF0025', 0.0, 0.014925124999999984, 0.043670498381265625],
 ['OFF0019,OFF0004,OFF0005', 0.0, 0.80199500000000001, 0.0],
 ['OFF0044,OFF0004,OFF0042', 'OFF0044', 0.97524937499999997, 0.0]]

## 畫圖來看分佈相似程度